# Convert the Excel interaction files to json for interpretation

This notebook walks through the handful of steps we want to take to convert the Excel files from dataset 4 to be interpretable JSON files.

Each Excel file will need to be manually modified before this can iterate over them. 
 - Make sure that the actions' column is clean and a single action per line (no `expand, read` combined events). Split any multiple events into two rows.
 - The content of 'action_details' column is a valid JSON object. That means it should look like `'key': 'value'` pairs. 
   - **query** should follow the pattern: `{"query_str": "_term_", "date_limits": ["yyyy-mm-dd", "yyyy-mm-dd"], "sort": {"_propName_": {"order": "desc | asc"}}}`
   - **query results** be an array like: `{"results": ["_list_, _of_, _results_"]}`
   - **expand**, **read**, **collapse** Events should be: `{"doc_id": "_id_"}`
   - Browser Search/Find functions should be type "**find**" and look like: `{"query_str": "_term_"}`
 - Do your best to fill in as many values in the time columns so that there is at least one column with a time for the entire session. The times are either in miliseconds or in video frames (out of 24). You may need to do some manual conversions.

## Set parameters

In [48]:
import pandas as pd
import json
import math

PID = 2
input_file = '../../data/Dataset_4/User Interactions_MannuallyModified/PandaJam_p2_shareable.xlsx'  # Replace with the path to your input Excel file
output_file = '../../data/Dataset_4/JSONInteractions/Panda_P'+str(PID)+'_interactions.json'  # Replace with the desired output JSON file path


## Import Data

Import Excel data and make a Python dictionary for the rest of the processing.

In [49]:
###
# Import file and convert to json
###
try:
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(input_file)
    # print(df)

    # Parse the JSON string into a Python dictionary
    data_list = df.to_dict(orient='records')
    print(data_list)
except Exception as e:
    print(f"Error: {str(e)}")

[{'time': '-00:01:39:82', 'video_timecode': '0:00:01.03', 'action': 'initialized', 'action_details': '{"page":"search"}', 'caption': nan, 'Percieved Action': nan, 'Notes': nan}, {'time': '00:00:01:13', 'video_timecode': '0:00:01.03', 'action': nan, 'action_details': nan, 'caption': 'P: All right, so now you want me to go ahead and go into the pilot?', 'Percieved Action': 'refers to questions', 'Notes': "P name is on the g-form with the intel questions. NOTE: any time the top of the g-form is shown, it has P's email address"}, {'time': '00:00:05:13', 'video_timecode': '0:00:05.03', 'action': nan, 'action_details': nan, 'caption': "E: Yeah if you're, if you're feeling, if you're feeling comfortable at least with the ", 'Percieved Action': nan, 'Notes': nan}, {'time': '00:00:09:85', 'video_timecode': '0:00:09.20', 'action': nan, 'action_details': nan, 'caption': "E: basic sort of questions and basic outline sure when you're ready,", 'Percieved Action': nan, 'Notes': nan}, {'time': '00:00:

## Lift Action_details up to Parent Object

To help with later processing, let's take the properties colapsed in a single column of the original Excel sheet into properties we can access for each event.

In [50]:
# Custom function to handle non-serializable values like NaN
def handle_non_serializable(obj):
    if isinstance(obj, float) and math.isnan(obj):
        return "NaN"
    else:
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

try:
    #Clean up nested properties that are in the Action Details column
    for obj in data_list:
        print(obj) #for testing
        if 'action_details' in obj:
            # Extract the 'details' object
            details = obj.pop('action_details')
            #if there is no object, leave it as None
            if isinstance(details, float):
                obj.update({'action_details': None})
            #if there is an object, lift it to the same level as the other properties
            else:
                obj.update(json.loads(details))

    print("lifted ActionDetails object properties to event level")

except Exception as e:
    print(f"Error: {str(e)}")

{'time': '-00:01:39:82', 'video_timecode': '0:00:01.03', 'action': 'initialized', 'action_details': '{"page":"search"}', 'caption': nan, 'Percieved Action': nan, 'Notes': nan}
{'time': '00:00:01:13', 'video_timecode': '0:00:01.03', 'action': nan, 'action_details': nan, 'caption': 'P: All right, so now you want me to go ahead and go into the pilot?', 'Percieved Action': 'refers to questions', 'Notes': "P name is on the g-form with the intel questions. NOTE: any time the top of the g-form is shown, it has P's email address"}
{'time': '00:00:05:13', 'video_timecode': '0:00:05.03', 'action': nan, 'action_details': nan, 'caption': "E: Yeah if you're, if you're feeling, if you're feeling comfortable at least with the ", 'Percieved Action': nan, 'Notes': nan}
{'time': '00:00:09:85', 'video_timecode': '0:00:09.20', 'action': nan, 'action_details': nan, 'caption': "E: basic sort of questions and basic outline sure when you're ready,", 'Percieved Action': nan, 'Notes': nan}
{'time': '00:00:13:83

## Designate think_aloud events
Look for events where the action column is empty but there is a string written in the caption cell and designate this as a **think_aloud** type of event.

In [51]:
###
# Create a thinkAloud event type for the dataset when there is nothing written as an action and some caption written 
###
for obj in data_list:
    if type(obj['action']) == float and type(obj['caption'] == str):
        obj.update({"action":"think_aloud"})


## Remove any NaN value
Finally, just to make valid JSON output, we need to handle any empty cells and set them to None (aka null in the final JSON output).

In [52]:
###
# Go through each event and replace NaN with Null
###
try:
    def replace_nan_with_none(obj):
        if isinstance(obj, list):
            return [replace_nan_with_none(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: replace_nan_with_none(value) for key, value in obj.items()}
        elif isinstance(obj, float) and math.isnan(obj):
            return None
        else:
            return obj

    cleaned_data = replace_nan_with_none(data_list)
    # print(cleaned_data)
except Exception as e:
    print(f"Error: {str(e)}")

## Export to file.

In [53]:
###
# Save to File
###
try:
    print("Total Number of events:",len(cleaned_data))
    # Save the data as a JSON file
    with open(output_file, 'w') as json_file:
        json.dump(cleaned_data, json_file, indent=4)

    print(f"Conversion successful. Output saved to {output_file}")
except Exception as e:
    print(f"Error: {str(e)}")

Total Number of events: 1406
Conversion successful. Output saved to ../../data/Dataset_4/JSONInteractions/Panda_P2_interactions.json
